In [2]:
import pandas as pd
import os,sys,re
import numpy as np
from collections import OrderedDict
os.getcwd()

'/Users/itaybendan/Covid19_forecasting'

In [22]:
from dateutil.parser import parse
parse("2/5/2019", dayfirst=True)
def dates_expr(d):
    return len(re.findall('[0-9]+',d)) >2 
def add_st(s):
    if len(s)==1:
        return "0"+s
    return s
"""
def convert_dates(dt):
    spl = dt.split("/")
    return spl[2]+"-"+add_st(spl[0])+"-"+add_st(spl[1])
"""
def convert_dates(dt):
    spl = parse(dt,dayfirst=False)
    return str(spl.year)+"-"+add_st(str(spl.month))+"-"+add_st(str(spl.day))

convert_dates("11/1/20"),convert_dates("24sep2020")

('2020-11-01', '2020-09-24')

In [23]:
stringency = pd.read_csv("/Users/itaybendan/covid-policy-tracker/data/timeseries/index_stringency.csv")
stringency.head()

,Unnamed: 0,Unnamed: 1,01jan2020,02jan2020,03jan2020,04jan2020,05jan2020,06jan2020,07jan2020,08jan2020,...,13oct2020,14oct2020,15oct2020,16oct2020,17oct2020,18oct2020,19oct2020,20oct2020,21oct2020,22oct2020
0,Aruba,ABW,0,0,0,0,0,0,0,0,...,50.45999908,50.45999908,50.45999908,50.45999908,50.45999908,50.45999908,50.45999908,50.45999908,50.45999908,.
1,Afghanistan,AFG,0,0,0,0,0,0,0,0,...,.,.,.,.,.,.,.,.,.,.
2,Angola,AGO,0,0,0,0,0,0,0,0,...,71.30000305,71.30000305,71.30000305,71.30000305,71.30000305,71.30000305,71.30000305,71.30000305,.,71.30000305
3,Anguilla,AIA,0,0,0,0,0,0,0,0,...,.,.,.,.,.,.,.,.,.,.
4,Albania,ALB,0,0,0,0,0,0,0,0,...,.,.,.,.,.,.,.,.,35.18999863,.


In [21]:
from datetime import datetime,date
from dateutil.parser import parse
days_diff =2
covid_data_path= "/Users/itaybd/covid19/COVID-19/new_covid/"
test_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
def dates_expr(d):
    if len(re.findall('[0-9]+',d))>=2: #(d.find("Unnamed") ==-1):
        print(d)
        print(parse(d))
    return len(re.findall('[0-9]+/[0-9]+/[0-9]+',d)) >0 and d == re.findall('[0-9]+/[0-9]+/[0-9]+',d)[0]
def add_st(s):
    if len(s)==1:
        return "0"+s
    return s
def convert_dates(dt):
    spl = dt.split("/")
    return spl[2]+"-"+add_st(spl[0])+"-"+add_st(spl[1])
print(datetime.now().strftime("%Y-%m-%d"))
dt1 = datetime.now().date()
test_file_path = os.path.join(covid_data_path,test_path)
#test_file_path ="/Users/itaybendan/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
test_file_path = "/Users/itaybendan/covid-policy-tracker/data/timeseries/index_stringency.csv"
if os.path.isfile(test_file_path):
    print("file exists")
    df_test = pd.read_csv(test_file_path)
    print(list(df_test.columns))
    date_cols = [c for c in df_test.columns if dates_expr(c)]
    dates_cnvrtd = [convert_dates(d) for d in date_cols]
    print(max(dates_cnvrtd))
    last_d = max(dates_cnvrtd)
    spl=last_d.split("-")
    ldt= date( 2000 +int(spl[0]),int(spl[1]),int(spl[2]))
    if days_diff < ( (dt1-ldt).days):
        #os.system("cd %s"%os.path.join(covid_data_path,"COVID-19"))
        os.chdir(os.path.join(covid_data_path,"COVID-19"))
        #print(os.getcwd())
        #print(os.listdir("."))
        os.system("git pull")
    else:
        print("up2date")
    


2020-11-02
file exists
['Unnamed: 0', 'Unnamed: 1', '01jan2020', '02jan2020', '03jan2020', '04jan2020', '05jan2020', '06jan2020', '07jan2020', '08jan2020', '09jan2020', '10jan2020', '11jan2020', '12jan2020', '13jan2020', '14jan2020', '15jan2020', '16jan2020', '17jan2020', '18jan2020', '19jan2020', '20jan2020', '21jan2020', '22jan2020', '23jan2020', '24jan2020', '25jan2020', '26jan2020', '27jan2020', '28jan2020', '29jan2020', '30jan2020', '31jan2020', '01feb2020', '02feb2020', '03feb2020', '04feb2020', '05feb2020', '06feb2020', '07feb2020', '08feb2020', '09feb2020', '10feb2020', '11feb2020', '12feb2020', '13feb2020', '14feb2020', '15feb2020', '16feb2020', '17feb2020', '18feb2020', '19feb2020', '20feb2020', '21feb2020', '22feb2020', '23feb2020', '24feb2020', '25feb2020', '26feb2020', '27feb2020', '28feb2020', '29feb2020', '01mar2020', '02mar2020', '03mar2020', '04mar2020', '05mar2020', '06mar2020', '07mar2020', '08mar2020', '09mar2020', '10mar2020', '11mar2020', '12mar2020', '13mar2020',

ValueError: max() arg is an empty sequence

In [4]:
def add_st(s):
    if len(s)==1:
        return "0"+s
    return s
def convert_dates(dt):
    spl = dt.split("/")
    return spl[2]+"-"+add_st(spl[0])+"-"+add_st(spl[1])

def convert_US(file_name):
    df = pd.read_csv(file_name)
    dfs = df.groupby("Province_State").sum()
    out1 = pd.read_csv("/Users/itaybd/output_covid/test_deathsNconfirmed_covid19.csv",index_col="index")
    dfs2 = dfs[list(dfs.columns[10:])]
    df_mod_t = dfs2.T
    df_mod_t.index = df_mod_t.apply(lambda r: convert_dates(r.name),axis=1 )
    DF_countries = {c:pd.DataFrame(df_mod_t[c]) for c in df_mod_t.columns}
    return DF_countries

In [17]:
def dates_expr(d):
    return len(re.findall('[0-9]+/[0-9]+/[0-9]+',d)) >0 and d == re.findall('[0-9]+/[0-9]+/[0-9]+',d)[0]

def create_df_dict(file_name,col_name,country_col='Country/Region',province_col='Province/State',rep_indexes=False,dict_indexes={}):
    df = pd.read_csv(file_name)
    df["key"] = df.apply(lambda r: str(r[country_col])+("_"+str(r[province_col])).replace("_nan",""),axis=1)
    #df1 = df.set_index("key")
    df = df.groupby("key").sum()
    date_cols = [c for c in df.columns if dates_expr(c)]   
    df_mod = df[date_cols]
    df_mod_t = df_mod.T
    df_mod_t.index = df_mod_t.apply(lambda r: convert_dates(r.name),axis=1 )
    DF_countries = {c:pd.DataFrame(df_mod_t[c]) for c in df_mod_t.columns}
    for c in DF_countries.keys():
        DF_countries[c] = DF_countries[c].rename(columns={c:col_name})
        if col_name.find("daily") == -1:
            #print("col_name is %s"%col_name)
            #print(DF_countries[c][col_name].shape)
            #print(DF_countries[c][col_name].tail())
            DF_countries[c]["daily_"+col_name] = DF_countries[c][col_name].diff(1).fillna(0)
        DF_countries[c]["Date"] = DF_countries[c].index
        DF_countries[c]["Country_Province"] = c 
        if rep_indexes:
            print(list(dict_indexes.keys())[-20:])
            DF_countries[c].index = DF_countries[c].apply(lambda r: dict_indexes[(c,r.name)],axis=1)
        else:
             DF_countries[c].index = DF_countries[c].apply(lambda r: (c,r.name),axis=1)
    return DF_countries

In [10]:
file_name = "/Users/itaybd/covid19/COVID-19/new_covid/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
dfs = convert_US(file_name)
dfs["California"].tail()

,California
20-10-13,16644
20-10-14,16729
20-10-15,16832
20-10-16,16910
20-10-17,16956


In [18]:
col_name="deaths"
start_col =20
D_x = create_df_dict(file_name,col_name,country_col='Country_Region',province_col='Province_State',rep_indexes=False,dict_indexes={})

list(D_x.keys())[-10:]

['US_Tennessee',
 'US_Texas',
 'US_Utah',
 'US_Vermont',
 'US_Virgin Islands',
 'US_Virginia',
 'US_Washington',
 'US_West Virginia',
 'US_Wisconsin',
 'US_Wyoming']

In [135]:
df = pd.read_csv("csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
df.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '10/8/20', '10/9/20', '10/10/20', '10/11/20', '10/12/20', '10/13/20',
       '10/14/20', '10/15/20', '10/16/20', '10/17/20'],
      dtype='object', length=274)

In [136]:
df["key"] = df.apply(lambda r: str(r['Country/Region'])+("_"+str(r['Province/State'])).replace("_nan",""),axis=1)
df = df.set_index("key")
df_mod = df[df.columns[4:]]

In [137]:
df_mod_t = df_mod.T
(df_mod_t[df_mod_t.columns[:4]]).head(),(df_mod_t[df_mod_t.columns[:4]]).tail()


(key      Afghanistan  Albania  Algeria  Andorra
 1/22/20            0        0        0        0
 1/23/20            0        0        0        0
 1/24/20            0        0        0        0
 1/25/20            0        0        0        0
 1/26/20            0        0        0        0,
 key       Afghanistan  Albania  Algeria  Andorra
 10/13/20         1480      429     1818       57
 10/14/20         1481      434     1827       59
 10/15/20         1481      439     1827       59
 10/16/20         1485      443     1841       59
 10/17/20         1488      448     1846       59)

In [138]:
DF_countries = {c:pd.DataFrame(df_mod_t[c]) for c in df_mod_t.columns}
DF_countries
type(DF_countries["Albania"]),DF_countries["Albania"].tail()
DF2 = DF_countries.copy()
def prepare_covid(gf,column_rep):
    for k in gf.keys():
        gf[k] = gf[k].rename(columns={k:column_rep})
        #gf[k]["daily_"+column_rep] = gf[k][column_rep].diff(1).fillna(0)
        gf[k].index = gf[k].apply(lambda r: (k,r.name),axis=1)
        
    df_all =  pd.concat(gf.values())#, keys=gf.keys())
    df_all  = df_all.sort_index()
    #L  = list(df_all.index.values)
    #E = list(enumerate(L))
    #D_real_indexes_vals = {x[1]:x[0] for x in E}
    #df_all.index = [x[0] for x in E]
    #return D_real_indexes_vals,df_all
    return df_all
    
#dict_indexes,DF3 = 
DF3 = prepare_covid(DF2,"deaths")
DF3.tail()

,deaths
"(Zimbabwe, 9/5/20)",206
"(Zimbabwe, 9/6/20)",206
"(Zimbabwe, 9/7/20)",210
"(Zimbabwe, 9/8/20)",218
"(Zimbabwe, 9/9/20)",222


In [139]:
import random,time
samp = random.sample(list(DF3.index.values),10000)

 
#DF3.index.values[-2:],DF3.apply(lambda r: r.name,axis=1)[-2:]

In [142]:
start = time.time()
#LL = [DF3.loc[x,"deaths"] for x in samp]
for x in samp:
    t = DF3.at[x,"deaths"]
    """
    if samp.index(x)%(500) == 0:
        print(x,t)
    """
     
round(time.time()-start,2)

4.24

In [143]:
import time
file_name = "csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
col_name = "deaths"
lags = [7,14,28,56,1]
lags2 = [7,14,28,56]

In [144]:
start = time.time()
def create_df_dict(file_name,col_name):
    df = pd.read_csv(file_name)
    df["key"] = df.apply(lambda r: str(r['Country/Region'])+("_"+str(r['Province/State'])).replace("_nan",""),axis=1)
    df = df.set_index("key")
    df_mod = df[df.columns[4:]]
    df_mod_t = df_mod.T
    DF_countries = {c:pd.DataFrame(df_mod_t[c]) for c in df_mod_t.columns}
    for c in DF_countries.keys():
        DF_countries[c] = DF_countries[c].rename(columns={c:col_name})
        if c=="Israel":
            print("Israel")
            print(DF_countries[c].columns)
        if col_name.find("daily") == -1:
            DF_countries[c]["daily_"+col_name] = DF_countries[c][col_name].diff(1).fillna(0)
        if c=="Israel":
            print("Israel")
            print(DF_countries[c].columns)
        DF_countries[c].index = DF_countries[c].apply(lambda r: (c,r.name),axis=1)
    return DF_countries
def add_lags(gf,lags,col_name):
    for c in gf.keys():
        
        for lg in lags:
            gf[c][col_name+"_lag_%d"%lg] = gf[c][col_name].shift(lg)
    return gf
round(time.time()-start,2)


0.02

In [145]:
def lag(df,col,lag,fillna_val = 0.0):
	return df[col].shift(lag).fillna(0.0)
def ma(df,col,window):
	return df[col].rolling(window).mean()
def sum(df,col,window):
    return df[col].rolling(window).sum()
def std(df,col,window):
	return df[col].rolling(window).std()
def pct_change(df,col,back):
	return df[col].pct_change(back).fillna(0.0)
def diff(df,col,back):
    return df[col].diff(back).fillna(0.0)
def local_cvt(df,col,back):
    return df[col].diff(back).fillna(0.0) -df[col].shift(back).fillna(0.0).diff(back).fillna(0.0)


In [146]:
def initialize_features_func_directional(lags,col_name):
    funcs_dict = OrderedDict()# OrderedDict()    
    for ii in lags:
        funcs_dict["%s_ma_%d"%(col_name,ii)] = {"func":ma,"params" : {"col":col_name,"window":ii}}
        funcs_dict["%s_sum_%d"%(col_name,ii)] = {"func":sum,"params" : {"col":col_name,"window":ii}}
        funcs_dict["%s_std_%d"%(col_name,ii)] = {"func":std,"params" : {"col":col_name,"window":ii}}
        #funcs_dict["%s_pct_%d"%(col_name,ii)] = {"func":pct_change,"params" : {"col":col_name,"back":ii}}
        funcs_dict["%s_diff_%d"%(col_name,ii)] = {"func":diff,"params" : {"col":col_name,"back":ii}}
        funcs_dict["%s_local_cvt_%d"%(col_name,ii)] = {"func":local_cvt,"params" : {"col":col_name,"back":ii}}
    return funcs_dict
def from_gf_to_df(frame,gf,col):
    #frame[col] = frame.apply(lambda r: gf[r.name[0]].at[r.name[1],col],axis=1)
    frame[col] = pd.concat([gf[k][col] for k in gf.keys()])


In [147]:
from tqdm import tqdm
def genetate_directional_features(frame,gf,func_dict,lags):
    func_dict_keys = list(func_dict.keys())
    for func_name in tqdm(func_dict_keys): #func_dict.keys():
            for kk in tqdm(gf.keys()):
                 params_d = func_dict[func_name]["params"].copy()
                 params_d["df"] = gf[kk]
                 
                 gf[kk][func_name] = func_dict[func_name]["func"](**params_d)
                 """
                 for jj in lags:
                    gf[kk][func_name+"__%d"%jj] = gf[kk][func_name].shift(jj).fillna(0)
                 """
            start2= time.time()
            from_gf_to_df(frame,gf,func_name)
            print("start2:",round(time.time()-start2,2))
            """
            for lg in lags:
                from_gf_to_df(frame,gf,func_name)
                #from_gf_to_df(frame,gf,func_name+"__%d"%lg)
            """

            

In [148]:
start = time.time()
DF_all = create_df_dict(file_name=file_name,col_name=col_name)
print(round(time.time()-start,2))
#DF_all["Israel"].tail()
DF_all["Israel"].at[("Israel","10/13/20"),"deaths"]

Israel
Index(['deaths'], dtype='object')
Israel
Index(['deaths', 'daily_deaths'], dtype='object')
3.89


2055

In [149]:
start = time.time()
functions_dict_deaths = initialize_features_func_directional(lags,"deaths")
print(round(time.time()-start,2))

0.01


In [151]:
print(DF3.at[("Israel","10/13/20"),"deaths"])
DF4 = DF3.copy()
DF4.columns
print(DF4.at[("Israel","10/13/20"),"deaths"])
DF4.loc[DF4.index.values[5222:5223],"deaths"]

2055
2055


(Bahrain, 4/14/20)    7
Name: deaths, dtype: int64

In [152]:

start = time.time()
DF_d = create_df_dict(file_name,col_name)
print(round(time.time()-start,2))
start = time.time()
DF_d = add_lags(gf=DF_d,lags= lags,col_name = col_name)
print(round(time.time()-start,2))
start = time.time()
#print(DF4.index[-5:])
#print(DF_d['nan_Zimbabwe'].tail())#,col_name+"_lag_7"])
for lg in lags:
    start2= time.time()
    from_gf_to_df(DF4,DF_d,col_name+"_lag_%d"%lg)
    print("start2:",round(time.time()-start2,2))
    
print(round(time.time()-start,2))


Israel
Index(['deaths'], dtype='object')
Israel
Index(['deaths', 'daily_deaths'], dtype='object')
5.1
5.33
start2: 0.33
start2: 0.25
start2: 0.16
start2: 0.25
start2: 0.15
1.15


In [154]:
DF_d["Israel"][-5:],DF4.at[("Israel","10/13/20"),"deaths"],DF4.at[("Israel","10/13/20"),"deaths_lag_7"]


(                    deaths  daily_deaths  deaths_lag_7  deaths_lag_14  \
 (Israel, 10/13/20)    2055          34.0        1797.0         1528.0   
 (Israel, 10/14/20)    2098          43.0        1824.0         1569.0   
 (Israel, 10/15/20)    2127          29.0        1864.0         1622.0   
 (Israel, 10/16/20)    2141          14.0        1886.0         1633.0   
 (Israel, 10/17/20)    2190          49.0        1941.0         1682.0   
 
                     deaths_lag_28  deaths_lag_56  deaths_lag_1  
 (Israel, 10/13/20)         1147.0          708.0        2021.0  
 (Israel, 10/14/20)         1165.0          781.0        2055.0  
 (Israel, 10/15/20)         1169.0          795.0        2098.0  
 (Israel, 10/16/20)         1196.0          809.0        2127.0  
 (Israel, 10/17/20)         1226.0          819.0        2141.0  ,
 2055,
 1797.0)

In [155]:
dict_deaths = initialize_features_func_directional(lags,"daily_deaths")
#print(list(dict_deaths.keys()))
print(len(list(dict_deaths.keys())))

25


In [156]:
start = time.time()
genetate_directional_features(DF4,DF_d,dict_deaths,lags)
print(round(time.time()-start,2))

 10%|█         | 28/267 [00:00<00:00, 272.44it/s]

start2: 0.23



  7%|▋         | 20/267 [00:00<00:01, 189.52it/s]

start2: 1.54



 11%|█         | 30/267 [00:00<00:00, 297.73it/s]

...

start2: 0.19



  8%|▊         | 22/267 [00:00<00:01, 217.44it/s]


start2: 0.11


100%|██████████| 25/25 [00:43<00:00,  1.76s/it]

start2: 0.12
44.06


In [174]:
print(DF4.at[("Israel","10/13/20"),"deaths"],DF4.at[("Israel","10/13/20"),"deaths_lag_7"]
)
#DF4.shape
AA = list(set(DF4.index.values))[-7:]
#print(list(set(DF4.index.values))[-7:])
#print(DF4.tail()[DF4.columns[:5]])
#print(DF4.loc[DF4.index.values[-3:]][DF4.columns[:5]])
print(DF4.at[("Israel","10/13/20"),"deaths"],DF4.at[("Israel","10/13/20"),"deaths_lag_7"]
)
print(DF4.at[("Germany","10/13/20"),"deaths"],DF4.at[("Germany","10/13/20"),"deaths_lag_7"]
)
print(("Israel","10/13/20") in DF4.index.values)
Az = [x for x in DF4.index.values if x[0] == "Israel" and x[1] == "10/13/20"][0]
print(Az)

print(Az == ("Israel","10/13/20"))
print(DF4.at[Az,"deaths"])
print(list(DF4.index.values).index(Az))
Av = DF4.index.values[list(DF4.index.values).index(Az)]
print(Av)
print(DF4.at[Av,"deaths"])
print(list(DF4.index.values).index(AA[4]))
print(DF4.at[AA[6],"deaths"])
#print(DF4.loc[AA[0]][DF4.columns[:5]])
"""
for c in DF4.columns:
    print (c,DF4.loc[AA][DF4.columns[:5]])
"""

#type(AA[0][1])
#print(AA[0][0],AA[0][1])
#print('Bhutan' == AA[0][0],'9/16/20' == AA[0][1],('Bhutan', '9/16/20')==AA[0])
#DF4.loc[AA[0],"deaths"]

2055 1797.0
2055 1797.0
9682 9566.0
False
('Israel', '10/13/20')
True
2055
39974
('Israel', '10/13/20')
2055
30915
665


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sys.path[0] == '':


'\nfor c in DF4.columns:\n    print (c,DF4.loc[AA][DF4.columns[:5]])\n'